In [1]:
class_names = {'question' : 0,'answer':1,'header':2,'other':3}

# Preprocess FUNSD for YOLO

In [4]:
import os
import json
import cv2
def main(json_dir,out_dir,image_dir):
    #  Read JSON file data
    for filename in os.listdir(json_dir):
      if filename.endswith(".json"):
        start = filename.split('.')[0]
        dir = os.path.join(json_dir,filename)
        with open(dir, 'r') as load_f:
            content = json.load(load_f)
            # print(content['form'])
            img_dir = os.path.join(image_dir,start+'.jpg')
            img = cv2.imread(img_dir,0)
            height = img.shape[0]
            width = img.shape[1]
            for t in content['form']: 
                filenm = out_dir + start + '.txt'
                if os.path.exists(filenm):
                    #  Calculate the relative x, y coordinate, W, H value of the center point required for YOLO data format
                    # xmin,ymin,xmax,ymax
                    x = (t['box'][0]+t['box'][2])/2
                    y = (t['box'][1]+t['box'][3])/2
                    w = t['box'][2]-t['box'][0]
                    h = t['box'][3]-t['box'][1]
                    x = x/width
                    y = y/height
                    w = w/width
                    h = h/height
 
                    with open(filenm, mode="r+", encoding="utf-8") as fp:
                      file_str = str(class_names[t['label']]) + ' ' + str(round(x, 6)) + ' ' + str(round(y, 6)) + ' ' + str(round(w, 6)) + \
                                ' ' + str(round(h, 6))
                      line_data = fp.readlines()

                      if len(line_data) != 0:
                          fp.write('\n' + file_str)
                      else:
                          fp.write(file_str)

                #  Create a file without existence
                else:
                    with open(filenm, mode="w", encoding="utf-8") as fp:
                      fp.close()

# Change the annotations to YOLO

In [5]:

from src.paths import *
# json_dir = '/content/drive/MyDrive/prodigal/dataset/training_data/annotations/'  # JSON file path
json_dir = os.path.join(FUNSD_TRAIN, 'adjusted_annotations/')
out_dir = os.path.join(FUNSD_TRAIN, 'labels/')  #  Output TXT file path
image_dir = os.path.join(FUNSD_TRAIN, 'images/')
main(json_dir,out_dir,image_dir)



In [6]:
# json_dir = '/content/drive/MyDrive/prodigal/dataset/training_data/annotations/'  # JSON file path
json_dir_test = os.path.join(FUNSD_TEST, 'adjusted_annotations/')
out_dir_test = os.path.join(FUNSD_TEST, 'labels/')  #  Output TXT file path
image_dir_test = os.path.join(FUNSD_TEST, 'images/')
main(json_dir_test,out_dir_test,image_dir_test)

# Convert PNG to JPG

In [7]:
import os
from PIL import Image
def convert_png_to_jpg(path):
  for filename in os.listdir(path):
      if filename.endswith(".png"):
        im1 = Image.open(os.path.join(path, filename))
        start = filename.split('.')[0]
        im1.save(os.path.join(path, start+'.jpg'))
        os.remove(os.path.join(path, filename))
      else:
          continue

In [8]:
convert_png_to_jpg(os.path.join(FUNSD_TRAIN, 'images/'))
convert_png_to_jpg(os.path.join(FUNSD_TEST, 'images/'))

# Training Script

In [11]:
! python ../src/models/yolov5/train.py --img 416 --batch 12 --epochs 1000 --data ../src/models/yolov5/data/data.yaml  --weights ../src/models/yolov5/weights/yolov5s.pt

train: weights=../src/models/yolov5/weights/yolov5-s.pt, cfg=, data=../src/models/yolov5/data/data.yaml, hyp=../src/models/yolov5/data/hyps/hyp.scratch-low.yaml, epochs=1000, batch_size=12, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=../src/models/yolov5/runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 316 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.1-289-g526e650 Python-3.9.12 torch-1.11.0+cu113 CUDA:0 (NVIDIA GeForce RTX 3090, 24268MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_ep

# Validation

In [2]:
! python ../src/models/yolov5/detect.py --img 1000 725 --source ../DATA/FUNSD/testing_data/images --weights ../src/models/checkpoints/backbone_yolo_single_class.pt --conf-thres 0.5 --save-txt

detect: weights=['../src/models/checkpoints/backbone_yolo_single_class.pt'], source=../DATA/FUNSD/testing_data/images, data=../src/models/yolov5/data/coco128.yaml, imgsz=[1000, 725], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=../src/models/yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-289-g526e650 Python-3.9.12 torch-1.11.0+cu113 CUDA:0 (NVIDIA GeForce RTX 3090, 24268MiB)

Fusing layers... 
Model summary: 213 layers, 7012822 parameters, 0 gradients
test_images/results/82092117.jpg
test_images/results/82092117.jpg
test_images/results/82092117.jpg
test_images/results/82092117.jpg
test_images/results/82092117.jpg
test_images/results/82092117.jpg
test_images/results/82092117.jpg
test_images/results/82092117.jpg
test_images/re